In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [3]:
class DSO:
	def __init__(self, totalHours, marginalCost, unitSalePrice):
		self.totalHours = totalHours
		self.numberOfAggregators = 0
		self.marginalCost = marginalCost
		self.unitSalePrice = unitSalePrice
		self.aggregators = []
		self.maxUtility = 0

	def addAggregator(self, aggregator):
		self.aggregators.append(aggregator)
		self.numberOfAggregators += 1

	def getAggregators(self):
		return self.aggregators

	def Sfunction(self, aggregator, tIndex):
		return aggregator.maximumDemand[tIndex] * self.maxPriceReference * (1 - np.exp(-aggregator.preferenceSatisfaction[tIndex] * (aggregator.actualLoad[tIndex] / aggregator.nominalDemand[tIndex])))

	def utilityFunction(self, m, theta, omega):
		sum = 0
		for i in range(self.numberOfAggregators):
			agg = self.aggregators[i]
			for j in range(self.totalHours):
				sum += self.unitSalePrice[j] * agg.actualLoad[j] - self.marginalCost[j] * agg.actualLoad[j] + omega * self.Sfunction(agg, j)
		sum -= theta*m
		return sum
	
	def optimalPrice(self):
		return self.utilityFunction(0, 0, 0)
		

class Aggregator:
	def __init__(self, totalHours, nominalDemand, minimumDemand, maximumDemand, maxPriceReference, preferenceSatisfaction, actualLoad):
		self.totalHours = totalHours
		self.nominalDemand = nominalDemand
		self.minimumDemand = minimumDemand
		self.maximumDemand = maximumDemand
		self.maxPriceReference = maxPriceReference
		self.preferenceSatisfaction = preferenceSatisfaction
		self.actualLoad = actualLoad
		self.currentUtility = 0
		self.bestLoadResponse = []

	def utilityFunction(self, DSO):
		sum = 0
		for i in range(DSO.totalHours):
			sum += DSO.SFunction(self, i) - DSO.unitSalePrice[i] * self.actualLoad[i]
		return sum

	def optimalDR(self, DSO):
		for j in range(DSO.totalHours):
			self.bestLoadResponse[j] = (DSO.nominalDemand[j] / self.preferenceSatisfaction[j]) * (np.log((self.preferenceSatisfaction[j] * self.maxPriceReference) / DSO.unitSalePrice[j]))


In [4]:
# dso = DSO()
# LAs = []

# for i in range(3):
#     LAs.append(Aggregator())